In [5]:
import sys
import importlib.metadata

REQUIRED_PACKAGES = {
    "opencv-python": "4.9.0.80",
    "numpy": "1.26.4",
    "mediapipe": "0.10.11"
}
REQUIRED_PACKAGES_NO_VERSION_DEPENDENCY = [
    "CodingRider",
    "keyboard"
]

print("환경 검증 시작...")

try:
    for package, required_version in REQUIRED_PACKAGES.items():
        installed_version = importlib.metadata.version(package)
        if not installed_version.startswith(required_version):
            raise ImportError(
                f"'{package}'의 버전이 잘못되었습니다. "
                f"필요 버전: {required_version}, 설치된 버전: {installed_version}"
            )
        print(f" - {package} {installed_version} (요구사항: {required_version}) - 확인 완료")
    for package in REQUIRED_PACKAGES_NO_VERSION_DEPENDENCY:
            installed_version = importlib.metadata.version(package)
            print(f" - {package} {installed_version} (설치 확인) - 확인 완료")
except importlib.metadata.PackageNotFoundError as e:
    raise type(e)(
        f"{e}\n\n"
        "필요한 패키지가 설치되지 않았습니다.\n"
        "VS Code 우측 상단의 커널 선택(Select Kernel) 버튼을 눌러 'mediapipe (0.10.11)'을 선택해주세요."
    ).with_traceback(e.__traceback__) from None
except ImportError as e:
    raise type(e)(
        f"{e}\n\n"
        "잘못된 커널이 선택되었습니다\n"
        "VS Code 우측 상단의 커널 선택(Select Kernel) 버튼을 눌러 'mediapipe (0.10.11)'을 선택해주세요."
    ).with_traceback(e.__traceback__) from None

print("모든 환경 검증 완료. 코드를 계속 실행하세요.")

환경 검증 시작...
 - opencv-python 4.9.0.80 (요구사항: 4.9.0.80) - 확인 완료
 - numpy 1.26.4 (요구사항: 1.26.4) - 확인 완료
 - mediapipe 0.10.11 (요구사항: 0.10.11) - 확인 완료
 - CodingRider 2.9 (설치 확인) - 확인 완료
 - keyboard 0.13.5 (설치 확인) - 확인 완료
모든 환경 검증 완료. 코드를 계속 실행하세요.


In [ ]:
# 포트 확인 후 알맞은 포트 이름을 입력해야 정상적으로 작동
# 예제는 실험 환경이었던 COM4로 진행
# 장치 관리자(Device manager)의 포트(Port) 항목에서 연결된 기기 확인 가능
OPEN_PORT = 'COM4'

In [ ]:
# openCV와 mediapipe를 이용한 손 트래킹 예제
import cv2
import mediapipe as mp

capture = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            continue 
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:            
            for hand_landmarks in results.multi_hand_landmarks: 
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
                
                # 검지 손가락을 선택합니다.
                finger = hand_landmarks.landmark[8] 

                # 검지 손가락의 좌표로 텍스트를 만듭니다.
                # finger.x는 x좌표, finger.y는 y좌표입니다.
                # 소수점 둘째 자리까지 나타냅니다.
                text = 'x : {:.2f}, y : {:.2f}'.format(finger.x, finger.y)
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 1)                
        cv2.imshow('MediaPipe Hand Detection', frame)
        if cv2.waitKey(1) == 27:
            break
        
capture.release()
cv2.destroyAllWindows()


In [ ]:
# 손으로 드론 조종하기
import cv2
import mediapipe as mp
from time import sleep
import keyboard
from CodingRider.drone import *
from CodingRider.protocol import *

# is_takeoff 변수로 이륙했는지 확인합니다.
is_takeoff = False

drone = Drone()
drone.open(OPEN_PORT)

capture = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            continue 
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:            
            for hand_landmarks in results.multi_hand_landmarks: 
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
                finger = hand_landmarks.landmark[8] 
                # 이륙했다면 검지 손가락 좌표로 드론을 움직입니다.   
                if is_takeoff:                
                    # 롤과 피치 값을 정합니다.
                    roll = int(((finger.x * 200) - 100) * 0.5)
                    pitch = -int(((finger.y * 200) - 100) * 0.5)
                    drone.sendControl(roll, pitch, 0, 0)

                    # 롤과 피치 값을 나타냅니다.    
                    text = f'r : {roll}, p : {pitch}'
                    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 1)                
        cv2.imshow('MediaPipe Hand Detection', frame)
        if keyboard.is_pressed('enter'):   
            print('이륙')       
            sleep(2)
            drone.sendTakeOff()
            sleep(5)
            print('이륙 완료') 
            is_takeoff = True
        if keyboard.is_pressed('space'):
            print('착륙')
            drone.sendControlWhile(0, 0, 0, 0, 500) # 제자리에서 0.5초 멈춥니다.
            drone.sendLanding()
            sleep(3)
            print('착륙 완료')
            is_takeoff = False
        if keyboard.is_pressed('q'):
            print('정지')
            drone.sendControlWhile(0, 0, 0, 0, 500)
            drone.sendStop()
            sleep(3)
            is_takeoff = False
        if cv2.waitKey(1) == 27:
            break
        
capture.release()
cv2.destroyAllWindows()